# 幾何プログラミングに基づく最適電力制御を模擬する(最適化模擬)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wadayama/MIKA2019/blob/master/powercontrol.ipynb)

## 必要なパッケージのインストール

* 凸最適化(幾何プログラミング)のためにcvxpyを利用します。インストールされていない場合は、"pip install cvxpy" としてインストールしてください。

In [17]:
import numpy as np
import torch
import torch.nn as nn  # ネットワーク構築用
import torch.optim as optim  # 最適化関数
import cvxpy as cp

## グローバル定数の設定

* 電力制御問題の詳細については、テキスト本文を参照してください。
* cvxpy のExample: https://www.cvxpy.org/examples/dgp/power_control.html をベースにして本プログラムを作成しています。

In [18]:
# Problem data
n = 5                     # number of transmitters and receivers
sigma = 0.5 * np.ones(n)  # noise power at the receiver i
p_min = 0.1 * np.ones(n)  # minimum power at the transmitter i
p_max = 5 * np.ones(n)    # maximum power at the transmitter i
sinr_min = 0.1            # threshold SINR for each receiver
G = np.array(  # パスゲイン行列
      [[1.0, 0.1, 0.2, 0.1, 0.05],
        [0.1, 1.0, 0.1, 0.1, 0.05],
        [0.2, 0.1, 1.0, 0.2, 0.2],
        [0.1, 0.1, 0.2, 1.0, 0.1],
        [0.05, 0.05, 0.2, 0.1, 1.0]]) 
mbs = 5 # ミニバッチサイズ
h = 100 # 隠れ層のユニット数
adam_lr = 0.01 # Adamの学習率

## ミニバッチ生成関数

* cvxpy を利用して電力制御問題を解く
* 入力パラメータは雑音分散
* 出力は最適電力配分

In [19]:
def gen_minibatch():
    sigma = np.random.rand(n)
    p = cp.Variable(shape=(n,), pos=True)
    objective = cp.Minimize(cp.sum(p))
    S_p = []
    for i in range(n):
        S_p.append(cp.sum(cp.hstack(G[i, k]*p for k in range(n) if i != k)))
    S = sigma + cp.hstack(S_p)
    signal_power = cp.multiply(cp.diag(G), p)
    inverse_sinr = S/signal_power
    constraints = [
        p >= p_min,
        p <= p_max,
        inverse_sinr <= (1/sinr_min),
    ]
    problem = cp.Problem(objective, constraints)
    problem.solve(gp=True)
    return torch.tensor(sigma).float(),  torch.tensor(p.value).float()

## ネットワークモデル

In [20]:
class Net(nn.Module): # nn.Module を継承
    def __init__(self): # コンストラクタ
        super(Net, self).__init__()
        self.comp = nn.Sequential(
            nn.Linear(n, h), 
            nn.ReLU(), # 活性化関数としてReLUを利用
            nn.Linear(h, h), 
            nn.ReLU(),
            nn.Linear(h, n), 
        )
    def forward(self, x): # 推論計算をforwardに書く
        x = self.comp(x)
        return x

## 訓練ループ

In [21]:
model = Net() # ネットワークインスタンス生成
loss_func = nn.MSELoss() # 損失関数の指定(二乗損失関数)
optimizer = optim.Adam(model.parameters(), lr=adam_lr) # オプティマイザの指定(Adamを利用)
for i in range(200):
    x, y = gen_minibatch() # ミニバッチの生成
    optimizer.zero_grad()  # オプティマイザの勾配情報初期化
    estimate = model(x)  # 推論計算
    loss = loss_func(y, estimate)  # 損失値の計算
    loss.backward()  # 誤差逆伝播法(後ろ向き計算の実行)
    optimizer.step()  # 学習可能パラメータの更新
    if i % 10 == 0:
      print('i =', i, 'loss =', loss.item())

i = 0 loss = 0.03914088010787964
i = 10 loss = 0.002022175583988428
i = 20 loss = 0.0023445826955139637
i = 30 loss = 0.0009369359468109906
i = 40 loss = 0.0010331157827749848
i = 50 loss = 0.0005970211932435632
i = 60 loss = 0.00034818227868527174
i = 70 loss = 0.0010689791524782777
i = 80 loss = 8.485699072480202e-05
i = 90 loss = 0.0001275963004445657
i = 100 loss = 0.00026759697357192636
i = 110 loss = 0.0005719877663068473
i = 120 loss = 0.0003433755482546985
i = 130 loss = 0.00022983089729677886
i = 140 loss = 0.00019253652135375887
i = 150 loss = 6.668653077213094e-05
i = 160 loss = 0.00038394492003135383
i = 170 loss = 8.533739310223609e-05
i = 180 loss = 0.00019873229030054063
i = 190 loss = 0.00013147920253686607


## 学習結果の確認


In [27]:
sigma, solution  = gen_minibatch()
print('sigma = ', sigma)
print('Solution    = ', solution)
print('NN solution = ', model(sigma))

sigma =  tensor([0.6829, 0.1197, 0.4287, 0.1138, 0.1542])
Solution    =  tensor([0.2068, 0.1419, 0.2027, 0.1541, 0.1496])
NN solution =  tensor([0.2117, 0.1283, 0.1898, 0.1605, 0.1435], grad_fn=<AddBackward0>)
